In [1]:
# Para que funcione, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [2]:
!echo $AWS_ACCESS_KEY_ID
!echo $AWS_SECRET_ACCESS_KEY
!echo $MLFLOW_S3_ENDPOINT_URL

$AWS_ACCESS_KEY_ID
$AWS_SECRET_ACCESS_KEY
$MLFLOW_S3_ENDPOINT_URL


In [3]:
# ========================== BLOQUE 1 ==========================
# 📌 CARGA DE DATOS Y BÚSQUEDA DE HIPERPARÁMETROS CON OPTUNA USANDO AUC
import os
import awswrangler as wr
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import optuna
import mlflow
import mlflow.sklearn
from sklearn.metrics import auc, roc_auc_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt

# 🚀 Cargar los datos desde AWS S3
X_train = wr.s3.read_csv("s3://data/final/train/neural_network/X_train_scaled-no_selection.csv")
y_train = wr.s3.read_csv("s3://data/final/train/neural_network/y_train.csv")
X_val = wr.s3.read_csv("s3://data/final/val/neural_network/X_val_scaled.csv-no_selection")
y_val = wr.s3.read_csv("s3://data/final/val/neural_network/y_val.csv")
X_test = wr.s3.read_csv("s3://data/final/test/neural_network/X_test_scaled-no_selection.csv")
y_test = wr.s3.read_csv("s3://data/final/test/neural_network/y_test.csv")

experiment_name = "Neural Networks AUC Threshold Optimization"
experiment_path = experiment_name.lower().replace(' ', '_')

# 🔹 Configurar MLflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment(experiment_name)

c:\Users\andre\workspace\andresmalvestiti-UBA-trabajo-final-experiments\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Experiment: artifact_location='s3://mlflow/24', creation_time=1741460965362, experiment_id='24', last_update_time=1741460965362, lifecycle_stage='active', name='Neural Networks AUC Threshold Optimization', tags={}>

In [4]:
import optuna
import tensorflow as tf
import mlflow
import mlflow.tensorflow
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

# Convertir DataFrames a arrays de NumPy y aplanar las series de etiquetas (si es necesario)
X_train = X_train.to_numpy()
y_train = y_train.to_numpy().ravel()
X_val   = X_val.to_numpy()
y_val   = y_val.to_numpy().ravel()
X_test  = X_test.to_numpy()
y_test  = y_test.to_numpy().ravel()

# 📌 Variable global para almacenar el mejor modelo
best_model = None
best_f1_score = 0  # Variable para rastrear el mejor F1-score

# 📌 Función para optimizar el threshold basado en F1-score
def optimize_threshold(y_true, y_pred_proba):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    f1_scores = [f1_score(y_true, (y_pred_proba >= t).astype(int), average="binary") for t in thresholds]
    return thresholds[np.argmax(f1_scores)]

# 📌 Función de objetivo para Optuna
def objective(trial):
    global best_model, best_f1_score

    # 🔹 Ajuste de hiperparámetros para evitar sobreajuste en un dataset pequeño
    neurons = trial.suggest_int("neurons", 16, 128, step=16)
    dropout_rate = trial.suggest_float("dropout", 0.3, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.01, log=True)
    
    # Regularización L2 para ayudar a prevenir el sobreajuste
    regularization = 0.001

    input_shape = (X_train.shape[1],)

    # 🔹 Crear modelo de red neuronal con menor complejidad y regularización
    model = Sequential([
        Dense(neurons, activation='relu', input_shape=input_shape, kernel_regularizer=l2(regularization)),
        BatchNormalization(),
        Dropout(dropout_rate),
        
        Dense(neurons // 2, activation='relu', kernel_regularizer=l2(regularization)),
        Dropout(dropout_rate),
        
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['AUC'])

    # 🔹 Early stopping con menos epochs para evitar sobreajuste
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # 🔹 Entrenar modelo con menos epochs
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=30,
                        batch_size=32,
                        callbacks=[early_stopping],
                        verbose=0)

    # 🔹 Obtener predicciones en validación
    y_pred_proba = model.predict(X_val).flatten()

    # 🔹 Calcular métricas básicas
    auc_score = roc_auc_score(y_val, y_pred_proba)
    optimal_threshold = optimize_threshold(y_val, y_pred_proba)
    y_pred = (y_pred_proba >= optimal_threshold).astype(int)
    f1 = f1_score(y_val, y_pred, average="binary")

    # 🔹 Calcular métricas adicionales a partir de la matriz de confusión
    cm = confusion_matrix(y_val, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = sensitivity  # Sensibilidad es igual a recall

    # 🔹 Graficar la curva ROC
    fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc_score:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    roc_fig = plt.gcf()
    mlflow.log_figure(roc_fig, "roc_curve.png")
    plt.close()

    # 🔹 Graficar la matriz de confusión
    fig, ax = plt.subplots()
    cax = ax.matshow(cm, cmap=plt.cm.Blues)
    fig.colorbar(cax)
    for (i, j), z in np.ndenumerate(cm):
        ax.text(j, i, str(z), ha='center', va='center')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    mlflow.log_figure(fig, "confusion_matrix.png")
    plt.close(fig)

    # 🔹 Registrar parámetros y métricas en MLflow en un run anidado
    with mlflow.start_run(nested=True):
        mlflow.log_param("neurons", neurons)
        mlflow.log_param("dropout", dropout_rate)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_metric("AUC", auc_score)
        mlflow.log_metric("F1_score", f1)
        mlflow.log_metric("Optimal_Threshold", optimal_threshold)
        mlflow.log_metric("Sensitivity", sensitivity)
        mlflow.log_metric("Specificity", specificity)
        mlflow.log_metric("Precision", precision)
        mlflow.log_metric("Recall", recall)
        # Guardar modelo si es el mejor hasta ahora (basado en validación)
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model
            model.save("best_neural_network.h5")
            mlflow.tensorflow.log_model(model, "model")
    return f1

# 🚀 Ejecutar optimización con Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# 🔹 Cargar el mejor modelo guardado
best_model = tf.keras.models.load_model("best_neural_network.h5")

# 🔹 Calcular métricas del mejor modelo en el set de prueba
y_pred_proba_test = best_model.predict(X_test).flatten()
auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
optimal_threshold_test = optimize_threshold(y_test, y_pred_proba_test)
y_pred_test = (y_pred_proba_test >= optimal_threshold_test).astype(int)
f1_test = f1_score(y_test, y_pred_test, average="binary")
cm_test = confusion_matrix(y_test, y_pred_test)
tn, fp, fn, tp = cm_test.ravel()
sensitivity_test = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity_test = tn / (tn + fp) if (tn + fp) > 0 else 0
precision_test = tp / (tp + fp) if (tp + fp) > 0 else 0
recall_test = sensitivity_test  # Sensibilidad es igual a recall

# 🔹 Graficar la curva ROC del mejor modelo en test
fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_proba_test)
plt.figure()
plt.plot(fpr_test, tpr_test, label=f'ROC curve (AUC = {auc_score_test:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Top Model')
plt.legend(loc='lower right')
roc_fig_test = plt.gcf()
plt.close()

# 🔹 Graficar la matriz de confusión del mejor modelo en test
fig, ax = plt.subplots()
cax = ax.matshow(cm_test, cmap=plt.cm.Blues)
fig.colorbar(cax)
for (i, j), z in np.ndenumerate(cm_test):
    ax.text(j, i, str(z), ha='center', va='center')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Top Model')
cm_fig_test = fig
plt.close(fig)

# Finalizar cualquier run activo antes de iniciar el run del top 1
if mlflow.active_run() is not None:
    mlflow.end_run()

# 🔹 Registrar en MLflow el top 1 (mejor modelo) con un identificador especial
with mlflow.start_run(run_name="Top_1_Model"):
    mlflow.log_param("neurons", study.best_trial.params["neurons"])
    mlflow.log_param("dropout", study.best_trial.params["dropout"])
    mlflow.log_param("learning_rate", study.best_trial.params["learning_rate"])
    mlflow.log_metric("AUC", auc_score_test)
    mlflow.log_metric("F1_score", f1_test)
    mlflow.log_metric("Optimal_Threshold", optimal_threshold_test)
    mlflow.log_metric("Sensitivity", sensitivity_test)
    mlflow.log_metric("Specificity", specificity_test)
    mlflow.log_metric("Precision", precision_test)
    mlflow.log_metric("Recall", recall_test)
    mlflow.log_figure(roc_fig_test, "roc_curve_top_model.png")
    mlflow.log_figure(cm_fig_test, "confusion_matrix_top_model.png")
    mlflow.tensorflow.log_model(best_model, "model_top1")

print("\n✅ Optimización completada. El mejor modelo se ha guardado y registrado en MLflow como Top_1_Model.")


[I 2025-03-10 22:58:44,332] A new study created in memory with name: no-name-59aee888-02fc-4fba-b243-01976188031e


4/4 [==============================] - 0s 3ms/step


2025/03/10 22:58:51 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\andre\AppData\Local\Temp\tmpavitk_sx\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\andre\AppData\Local\Temp\tmpavitk_sx\model\data\model\assets
2025/03/10 22:59:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/03/10 22:59:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run ambitious-rook-184 at: http://localhost:5000/#/experiments/24/runs/47845a4cffef40f598915db0bb426d5b.
2025/03/10 22:59:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/24.
[I 2025-03-10 22:59:07,774] Trial 0 finished with value: 0.6 and parameters: {'neurons': 48, 'dropout': 0.3684185157439498, 'learning_rate': 0.0035762910932102695}. Best is trial 0 with value: 0.6.


...

4/4 [==============================] - 0s 1ms/step


2025/03/10 23:02:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run amazing-bass-952 at: http://localhost:5000/#/experiments/24/runs/bd509c51b8cd40cab57121aef23bcefb.
2025/03/10 23:02:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/24.
2025/03/10 23:02:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\andre\AppData\Local\Temp\tmp_erhwxei\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\andre\AppData\Local\Temp\tmp_erhwxei\model\data\model\assets
2025/03/10 23:03:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/03/10 23:03:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run Top_1_Model at: http://localhost:5000/#/experiments/24/runs/3e3faa7c1b184bc1bbc822190d4f5512.
2025/03/10 23:03:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/24.



✅ Optimización completada. El mejor modelo se ha guardado y registrado en MLflow como Top_1_Model.
